In [1]:
import torch.utils.data
from transformers import BartConfig, Seq2SeqTrainingArguments, IntervalStrategy, SchedulerType, TrainingArguments


from data.dataset import init_dataset

/home/user1-selab3/miniconda3/envs/myenv_python3_11/lib/python3.11/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


Checking the attributes of pre_train pickle file

In [2]:
import pickle

filename = '/home/user1-selab3/Documents/pre_train.pk'

with open(filename, 'rb') as file:
    dataset = pickle.load(file)

attributes = dir(dataset)
print("Attributes in dataset:", attributes)

data_attributes = [attr for attr in attributes if not attr.startswith('__') and not callable(getattr(dataset, attr))]
print("Data-related attributes in dataset:", data_attributes)


Attributes in dataset: ['EOS_TOKEN', 'MSK_TOKEN', 'PAD_TOKEN', 'SEP_TOKEN', 'SOS_TOKEN', 'START_VOCAB', 'UNK_TOKEN', '_Vocab__special_symbols', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_special_symbols', 'decode', 'decode_batch', 'encode_batch', 'encode_sequence', 'eos_processor', 'get_eos_index', 'get_index', 'get_mask_index', 'get_pad_index', 'get_sos_index', 'get_token', 'get_unk_index', 'ignore_case', 'index_offset', 'method', 'name', 'num_special_token', 'pad_token_id', 'restore_index', 'save', 'save_pickle', 'save_pretrained', 'sep_processor', 'sos_processor', 'tokenizer', 'transfer_index']
Data-related attributes in dataset: ['EOS_TOKE

In [41]:
pickle_file_path = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset_backup/dataset_saved/fine_tune.search.java.train.pk'#fine_tune.completion.java.test.pk'

with open(pickle_file_path, 'rb') as file:
    dataset = pickle.load(file)

# print(dataset.dataset_dir)
# print(dataset.args)
# print(dataset.asts)

print(dataset.codes[:1])
print(len(dataset.codes))


print(dataset.asts[:1])
print(len(dataset.asts))

# print(dataset)

# first_five_items = dataset[:2]

# for item in first_five_items:
#     print(item)


['@ Override public ImageSource apply ( ImageSource input ) { final int [ ] [ ] pixelMatrix = new int [ 3 ] [ 3 ] ; int w = input . getWidth ( ) ; int h = input . getHeight ( ) ; int [ ] [ ] output = new int [ h ] [ w ] ; for ( int j = 1 ; j < h - 1 ; j ++ ) { for ( int i = 1 ; i < w - 1 ; i ++ ) { pixelMatrix [ 0 ] [ 0 ] = input . getR ( i - 1 , j - 1 ) ; pixelMatrix [ 0 ] [ 1 ] = input . getRGB ( i - 1 , j ) ; pixelMatrix [ 0 ] [ 2 ] = input . getRGB ( i - 1 , j + 1 ) ; pixelMatrix [ 1 ] [ 0 ] = input . getRGB ( i , j - 1 ) ; pixelMatrix [ 1 ] [ 2 ] = input . getRGB ( i , j + 1 ) ; pixelMatrix [ 2 ] [ 0 ] = input . getRGB ( i + 1 , j - 1 ) ; pixelMatrix [ 2 ] [ 1 ] = input . getRGB ( i + 1 , j ) ; pixelMatrix [ 2 ] [ 2 ] = input . getRGB ( i + 1 , j + 1 ) ; int edge = ( int ) convolution ( pixelMatrix ) ; int rgb = ( edge << 16 | edge << 8 | edge ) ; output [ j ] [ i ] = rgb ; } } MatrixSource source = new MatrixSource ( output ) ; return source ; }']
164273
['local_variable_declarat

To Check and print information about Codes and Asts if available

In [7]:

print("Total dataset length:", len(dataset))

#Java Code lines index
index_start = 0
index_end = 1000

if hasattr(dataset, 'asts') and isinstance(dataset.asts, list):
    sliced_asts = dataset.asts[index_start:index_end]  # Adjust indices as needed
    print("Length of sliced AST data:", len(sliced_asts))
    if sliced_asts:
        print("Sample AST from sliced data:", sliced_asts[0])
    else:
        print("Sliced AST data is empty.")
else:
    print("The 'asts' attribute does not exist or is not a list.")


if hasattr(dataset, 'codes'):
    if isinstance(dataset.codes, list):
        sliced_codes = dataset.codes[index_start:index_end]  # Adjust indices as needed
        print("Length of sliced Code data:", len(sliced_codes))
        if sliced_codes:
            print("Sample Code from sliced data:", sliced_codes[0])
        else:
            print("Sliced Code data is empty.")
    else:
        print("The 'codes' attribute exists but is not a list, it is:", type(dataset.codes))
else:
    print("The 'codes' attribute does not exist.")


Total dataset length: 19413
Length of sliced AST data: 1000
Sample AST from sliced data: local_variable_declaration local_variable_declaration for_statement__ local_variable_declaration binary_expression update_expression local_variable_declaration__ binary_expression__ parenthesized_expression__ binary_expression__ binary_expression __binary_expression __parenthesized_expression __binary_expression __local_variable_declaration local_variable_declaration__ object_creation_expression __local_variable_declaration local_variable_declaration if_statement__ parenthesized_expression__ binary_expression __parenthesized_expression return_statement __if_statement __for_statement throw_statement__ object_creation_expression__ binary_expression __object_creation_expression __throw_statement
Length of sliced Code data: 1000
Sample Code from sliced data: file getFile ( string dirsProp , string path ) throws ioException { string [ ] dirs = getTrimmedStrings ( dirsProp ) ; int hashCode = path . hashC

Saving the Asts and corresponding codes to jsonl file

In [5]:
if hasattr(dataset, 'asts') and hasattr(dataset, 'codes') and dataset.asts and dataset.codes:
    if len(dataset.asts) != len(dataset.codes):
        print("The lengths of 'asts' and 'codes' do not match.")
    else:
        ast_code_pairs = list(zip(dataset.asts, dataset.codes))
        
        unique_ast_code_pairs = set(ast_code_pairs)
        
        with open('unique_ast_code_pairs.jsonl', 'w') as file:
            file.write("Unique ASTs with corresponding Codes:\n")
            for ast, code in unique_ast_code_pairs:
                file.write(f"AST: {ast}\nCode: {code}\n\n")

        print("Unique AST and Code pairs have been written to 'unique_ast_code_pairs.jsonl'")
else:
    print("Either 'asts' or 'codes' attribute is empty or does not exist.")


Unique AST and Code pairs have been written to 'unique_ast_code_pairs.txt'


Checking Unique components in finetuning files:

In [2]:
import pickle
#filename = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset_backup/dataset_saved/fine_tune.search.java.valid.pk'
filename = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset_backup/dataset_saved/pre_train.pk'

with open(filename, 'rb') as file:
    dataset = pickle.load(file)
    print(len(dataset))
    
if hasattr(dataset, 'asts'):
    print("Type of 'asts':", type(dataset.asts))
    if isinstance(dataset.asts, list) and len(dataset.asts) > 0:
        print("Sample of 'asts':", dataset.asts[0])
    else:
        print("The 'asts' list is empty.")
else:
    print("The 'asts' attribute does not exist.")

if hasattr(dataset, 'asts') and dataset.asts:
    extracted_data = ' '.join(str(ast) for ast in dataset.asts)
    components = extracted_data.split()  
    unique_components = list(set(components))  
    unique_components.sort() 

    print("Unique components in 'asts':")
    for component in unique_components:
        print(component)
else:
    print("The 'asts' attribute is empty or does not exist.")

654393
Type of 'asts': <class 'list'>
Sample of 'asts': short_var_declaration__ unary_expression__ call_expression__ selector_expression __call_expression __unary_expression __short_var_declaration for_statement__ call_expression __for_statement if_statement__ binary_expression__ call_expression__ selector_expression __call_expression __binary_expression short_var_declaration assignment_statement__ parameter_declaration call_expression__ selector_expression selector_expression __call_expression return_statement__ call_expression __return_statement __assignment_statement __if_statement return_statement__ call_expression__ call_expression __call_expression __return_statement
Unique components in 'asts':
__anonymous_function_creation_expression
__array_creation_expression
__assert_statement
__assignment
__assignment_expression
__assignment_statement
__augmented_assignment
__augmented_assignment_expression
__await_expression
__binary_expression
__call_expression
__call_statement
__case_sta

In [1]:
# Open the two source files and the output file
with open('/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/spt-code/sources/data/asts/ast_jdt/longmethod.txt', 'r') as source_file, \
     open('/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/spt-code/sources/data/asts/ast_jdt/ast_outputs_pretrain.txt', 'r') as asts_file, \
     open('/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/asts/ast_jdt_nongen_output_file.txt', 'w') as output_file:

    # Read lines from both files
    source_lines = source_file.readlines()
    asts_lines = asts_file.readlines()

    # Iterate over the lines from both files
    for source_line, asts_line in zip(source_lines, asts_lines):
        # Strip newline characters and concatenate the lines with the desired format
        source_line_clean = source_line.replace('"', '').strip()
        asts_line_clean = asts_line.strip()

        formatted_line = f'Source: "{source_line_clean}", Asts: "{asts_line_clean}"\n'
        
        # Write the formatted line to the output file
        output_file.write(formatted_line)

print("The files were concatenated successfully and saved to ast_jdt_nongen_output_file.")


The files were concatenated successfully and saved to ast_jdt_nongen_output_file.


Concat cleaned method and their respective ast into a file since java is not working

In [ ]:
# Open the two source files and the output file
with open('/home/user1-system11/Documents/data_shradha/asts/pretrain/pretrain_source.txt', 'r') as source_file, \
     open('/home/user1-system11/Documents/data_shradha/spt-code-new-data/pretrain_methods_asts_blockexpr.txt', 'r') as asts_file, \
     open('/home/user1-system11/Documents/data_shradha/asts/pretrain_ast_jdt_output', 'w') as output_file:

    # Read lines from both files
    source_lines = source_file.readlines()
    asts_lines = asts_file.readlines()

    # Iterate over the lines from both files
    for source_line, asts_line in zip(source_lines, asts_lines):
        # Strip newline characters and concatenate the lines with the desired format
        source_line_clean = source_line.replace('"', '').strip()
        asts_line_clean = asts_line.strip()

        formatted_line = f'Source: "{source_line_clean}", Asts: "{asts_line_clean}"\n'
        
        # Write the formatted line to the output file
        output_file.write(formatted_line
                          )

print("The files were concatenated successfully and saved to finetune_methods_test.txt.")


The files were concatenated successfully and saved to finetune_methods_test.txt.


To prepare a finetuning dataset:

In [6]:
import json

# Path to your JSON file
file_path = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset/finetune_raw/java-small-json/java-small.val.json'

# Open and read the file line by line (assuming each line is a separate JSON object)
results = []
with open(file_path, 'r') as file:
    for line in file:
        try:
            # Parse each line as a JSON object
            json_object = json.loads(line)
            # Format the desired fields
            formatted_string = f"{json_object['left_context']} {json_object['target_seq']} {json_object['right_context']}"
            results.append(formatted_string)
        except json.JSONDecodeError:
            print("Error decoding JSON from line:", line)
            continue

# Write all formatted strings to a new file
with open('finetune_raw_valid.txt', 'w') as output_file:
    for item in results:
        output_file.write(item + '\n')

print("Data has been processed and saved to finetune_raw_valid.txt.")



Data has been processed and saved to finetune_raw_valid.jsonl.


In [2]:
import json
import re

# Regular expression pattern to match string literals
STRING_MATCHING_PATTERN = re.compile(r'([bruf]*)(\"\"\"|\'\'\'|\"|\')(?:(?!\2)(?:\\.|[^\\]))*\2')

def replace_string_literal(source):
    """Replace string literals in the source code with '___STR'."""
    return re.sub(pattern=STRING_MATCHING_PATTERN, repl='___STR', string=source)

# Path to your JSON file
file_path = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset/finetune_raw/java-small-json/java-small.val.json'
output_file_path = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset/finetune_raw/java-small-json/finetune_methods_valid_final.txt'

results = []  # Store the output lines here

# Open and read the file line by line (assuming each line is a separate JSON object)
with open(file_path, 'r') as file:
    line_count = 0  # Counter for input lines
    for line in file:
        line_count += 1
        try:
            # Parse each line as a JSON object
            json_object = json.loads(line)
            # Replace string literals and concatenate the desired fields
            left_context = replace_string_literal(json_object['left_context']).replace('\n', '\\n').replace('=', '\\u003d').replace("\\", "\\\\").replace("\"", "\\\"").replace("\r", "\\r").replace("\t", "\\t")
            right_context = replace_string_literal(json_object['right_context']).replace('\n', '\\n').replace('=', '\\u003d').replace("\\", "\\\\").replace("\"", "\\\"").replace("\r", "\\r").replace("\t", "\\t")
            target_seq = replace_string_literal(json_object['target_seq']).replace('\n', '\\n').replace('=', '\\u003d').replace("\\", "\\\\").replace("\"", "\\\"").replace("\r", "\\r").replace("\t", "\\t")
            formatted_string = f"{left_context} {target_seq} {right_context}"
            # Enclose the concatenated string in quotes
            enclosed_string = f"\"{formatted_string}\""
            results.append(enclosed_string)
        except json.JSONDecodeError:
            # Log the error and append a placeholder line
            print(f"Error decoding JSON on input line {line_count}")
            results.append(f"\"Error in JSON format on line {line_count}\"")

# Print the line counts
print(f"Total input lines read: {line_count}")
print(f"Total output lines prepared: {len(results)}")

# Write all prepared lines to the output file
with open(output_file_path, 'w') as output_file:
    for item in results:
        output_file.write(item + '\n')

print(f"Data has been processed and saved to {output_file_path}.")


Total input lines read: 10000
Total output lines prepared: 10000
Data has been processed and saved to /home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset/finetune_raw/java-small-json/finetune_methods_valid_pred_final.txt.


In [1]:
# Define the path to the JSON file
file_path = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset/finetune_raw/java-small-json/java-small.train.json'

# Initialize a counter for the lines
line_count = 0

# Open the file and count the lines
with open(file_path, 'r') as file:
    for line in file:
        line_count += 1

# Print the number of lines in the file
print(f'The file has {line_count} lines.')


The file has 1309842 lines.


In [5]:
# if hasattr(dataset, 'asts') and hasattr(dataset, 'codes') and dataset.asts and dataset.codes:
#     if len(dataset.asts) != len(dataset.codes):
#         print("The lengths of 'asts' and 'codes' do not match.")
#     else:
#         ast_code_pairs = list(zip(dataset.asts, dataset.codes))
        
#         unique_ast_code_pairs = set(ast_code_pairs)
        
#         print("Unique ASTs with corresponding Codes:")
#         for ast, code in unique_ast_code_pairs:
#             print("AST:", ast, "Code:", code)
# else:
#     print("Either 'asts' or 'codes' attribute is empty or does not exist.")


In [ ]:
# import pickle

# filename = '/home/user1-selab3/Documents/pre_train.pk'
# # filename = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset/dataset_saved/pre_train.pk'

# with open(filename, 'rb') as file:
#     dataset = pickle.load(file)
#     print(len(dataset))

# if hasattr(dataset, 'asts') and isinstance(dataset.asts, list) and len(dataset.asts) >= 226483:
#     sliced_data = dataset.asts[45423:226484]  

#     print("Length of sliced data:", len(sliced_data))
#     if sliced_data:
#         print("Sample of 'asts' from sliced data:", sliced_data[0])
#     else:
#         print("Sliced data is empty.")
# else:
#     print("The 'asts' attribute does not exist or does not contain enough data.")

# if hasattr(dataset, 'asts') and dataset.asts:
#     extracted_data = ' '.join(str(ast) for ast in dataset.asts)
#     components = extracted_data.split() 
#     unique_components = list(set(components))  
#     unique_components.sort()  

#     print("Unique components in 'asts':")
#     for component in unique_components:
#         print(component)
# else:
#     print("The 'asts' attribute is empty or does not exist.")